In [13]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [14]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [15]:
train_filename = "/kaggle/input/datasetmovielens/train_movielens_1m.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "/kaggle/input/datasetmovielens/test_movielens_1m.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_movie_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_movie_rating': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_movie_year': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(1980, 10)),
    'context_movie_genre': tf.io.FixedLenFeature([10], tf.string, default_value=np.repeat("Drama", 10)),
    'label_movie_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})


In [16]:
movies = tfds.load("movielens/1m-movies", split='train')
movies = movies.map(lambda x: x["movie_id"])
movie_ids = movies.batch(1_000)
unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))

In [17]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_movie_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension), 
    tf.keras.layers.GRU(embedding_dimension),
])

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_movie_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension)
])

metrics = tfrs.metrics.FactorizedTopK(candidates=movies.batch(128).map(candidate_model))

task = tfrs.tasks.Retrieval(metrics=metrics)

class Model(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model
        self._task = task

    def compute_loss(self, features, training=False):
        watch_history = features["context_movie_id"]
        watch_next_label = features["label_movie_id"]

        query_embedding = self._query_model(watch_history)       
        candidate_embedding = self._candidate_model(watch_next_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model = Model(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()

In [40]:
history = model.fit(cached_train, epochs=10,callbacks=[es],validation_data=cached_test)

Epoch 1/10
67/67 [==============================] - 35s 525ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 92641.6588 - regularization_loss: 0.0000e+00 - total_loss: 92641.6588 - val_factorized_top_k/top_1_categorical_accuracy: 0.0146 - val_factorized_top_k/top_5_categorical_accuracy: 0.0786 - val_factorized_top_k/top_10_categorical_accuracy: 0.1375 - val_factorized_top_k/top_50_categorical_accuracy: 0.3874 - val_factorized_top_k/top_100_categorical_accuracy: 0.5305 - val_loss: 9433.8848 - val_regularization_loss: 0.0000e+00 - val_total_loss: 9433.8848
Epoch 2/10
67/67 [==============================] - 46s 690ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e

In [41]:
prediction = model.evaluate(cached_test, return_dict=True)

37/37 [==============================] - 30s 815ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0787 - factorized_top_k/top_10_categorical_accuracy: 0.1373 - factorized_top_k/top_50_categorical_accuracy: 0.3873 - factorized_top_k/top_100_categorical_accuracy: 0.5308 - loss: 15418.5656 - regularization_loss: 0.0000e+00 - total_loss: 15418.5656


In [49]:
prediction

{'factorized_top_k/top_1_categorical_accuracy': 0.014275205321609974,
 'factorized_top_k/top_5_categorical_accuracy': 0.07870019972324371,
 'factorized_top_k/top_10_categorical_accuracy': 0.13728290796279907,
 'factorized_top_k/top_50_categorical_accuracy': 0.38730689883232117,
 'factorized_top_k/top_100_categorical_accuracy': 0.5308265686035156,
 'loss': 9433.9912109375,
 'regularization_loss': 0,
 'total_loss': 9433.9912109375}